In [1]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS

In [2]:
import tkinter as tk
from tkinter import filedialog

def pick_file():
    root = tk.Tk()
    root.withdraw()
    file_path = filedialog.askopenfilename()
    return file_path

# location of the pdf file/files. 
# reader = pick_file()

In [11]:
import PyPDF2

# Open the PDF file in read-binary mode
with open(pick_file(), 'rb') as file:
    # Create a PyPDF2 PdfFileReader object
    reader = PyPDF2.PdfReader(file)

    # Read the text from each page and append it to the raw_text variable
    raw_text = ''
    for i in range(len(reader.pages)):
        page = reader.pages[i]
        text = page.extract_text()
        if text:
            raw_text += text


In [ ]:
raw_text[:100]

In [12]:
# We need to split the text that we read into smaller chunks so that during information retreival we don't hit the token size limits. 

text_splitter = CharacterTextSplitter(        
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [ ]:
len(texts)

In [ ]:
texts[0]

In [13]:
# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings()

In [14]:
docsearch = FAISS.from_texts(texts, embeddings)

In [ ]:
docsearch

In [15]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [16]:
chain = load_qa_chain(OpenAI(), chain_type="stuff")

In [17]:
query = input("Question: ")
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' UPS specific format "PLD0200".'